In [17]:
crimes <- read.csv("../data/crimes.csv")
crimes$population = crimes$density * crimes$area
crimes$urban = crimes$population > 50000
head(crimes)

crimes,prbarr,prbpris,polpc,density,area,taxpc,region,pctmin,pctymale,wcon,wsta,wser,wtrd,wfir,name,population,urban
3901,0.289696,0.472222,0.0017868,230.71590,423.943,25.69763,central,20.21870,0.0876968,206.4803,236.24,215.7335,182.3330,272.4492,Alamance County,97810.391,TRUE
416,0.202899,0.465753,0.0005939,97.68340,259.994,14.56088,central,7.91632,0.0870046,188.7683,247.38,191.3742,151.4234,202.4292,Alexander County,25397.098,FALSE
91,0.406593,0.500000,0.0008209,41.27659,235.059,18.63060,west,3.16053,0.0738525,147.9290,233.33,158.2278,143.5132,200.3205,Alleghany County,9702.434,FALSE
562,0.431095,0.431373,0.0014327,48.21764,531.452,38.24473,central,47.91610,0.0836378,284.5809,206.07,208.4636,174.2457,207.1006,Anson County,25625.361,FALSE
170,0.631579,0.383333,0.0008353,53.05164,426.135,16.85321,west,1.79619,0.0790580,195.9361,234.75,178.7785,152.7354,209.0301,Ashe County,22607.161,FALSE
275,0.369650,0.226415,0.0019067,59.10931,247.087,22.32144,west,1.54070,0.1063334,199.2377,227.69,176.5392,150.5835,195.3125,Avery County,14605.142,FALSE


In [2]:
floor(runif(3, min = 0, max = 10000))

[1]  769 5516   28

In [13]:
d_cross_validation <- function(d, dataset, formulas) {
    index <- rep(1:d, length.out=nrow(dataset))
    index <- sample(index)
    
    sep_list <- rep(0, length.out=length(formulas))
    
    for (i in 1:d) {
        test_set <- dataset[index==i, ]
        train_set <- dataset[index!=i, ]
        
        m_list <- lapply(formulas, function(formula) {glm(formula, data=train_set, family=poisson(link="log"))})
        
        for (j in 1:length(m_list)){
            model <- m_list[[j]]
            pred <- predict(model, newdata=test_set, type="response")
            sep_list[j] <- sep_list[j] + sum((test_set$crimes - pred)^2)
        }
    }
    
    msep_list = sep_list / d
    rmsep_list = sqrt(msep_list)
    
    return(data.frame(SEP=sep_list, MSEP=msep_list, RMSEP=rmsep_list))
}
require(MASS)
d_cross_validation_nb <- function(d, dataset, formulas) {
    index <- rep(1:d, length.out=nrow(dataset))
    index <- sample(index)
    
    sep_list <- rep(0, length.out=length(formulas))
    
    for (i in 1:d) {
        test_set <- dataset[index==i, ]
        train_set <- dataset[index!=i, ]
        
        m_list <- lapply(formulas, function(formula) {glm.nb(formula, data=train_set)})
        
        for (j in 1:length(m_list)){
            model <- m_list[[j]]
            pred <- predict(model, newdata=test_set, type="response")
            sep_list[j] <- sep_list[j] + sum((test_set$crimes - pred)^2)
        }
    }
    
    msep_list = sep_list / d
    rmsep_list = sqrt(msep_list)
    
    return(data.frame(SEP=sep_list, MSEP=msep_list, RMSEP=rmsep_list))
}

# runs multiple experiments and averages over them
eval <- function(n, d, seed, dataset, formulas){
    set.seed(seed)
    seeds = floor(runif(n, min = 0, max = 10000))
    set.seed(seeds[1])
    df <- d_cross_validation(d, dataset, formulas)
    for (i in 2:n){
        set.seed(seeds[i])
        df = df + d_cross_validation(d, dataset, formulas)
    }
    return (df/n)
}
eval_nb <- function(n, d, seed, dataset, formulas){
    set.seed(seed)
    seeds = floor(runif(n, min = 0, max = 10000))
    set.seed(seeds[1])
    df <- d_cross_validation_nb(d, dataset, formulas)
    for (i in 2:n){
        set.seed(seeds[i])
        df = df + d_cross_validation_nb(d, dataset, formulas)
    }
    return (df/n)
}

In [14]:
eval_nb(100, 9, 5, crimes, c(
    as.formula("crimes ~ prbarr"),
    as.formula("crimes ~ prbarr + area"),
    as.formula("crimes ~ region"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ I(density*area) + region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ I(density*area) + region")))

SEP,MSEP,RMSEP
2.139223e+09,237691419,15416.50
2.118699e+09,235410991,15342.26
2.170531e+09,241170057,15528.77
1.274362e+09,141595774,11681.81
2.169210e+09,241023373,15310.99
3.451464e+09,383496011,19364.97
1.335529e+12,148392093183,359701.92


In [8]:
eval(100, 9, 5, crimes, c(
    as.formula("crimes ~ prbarr"),
    as.formula("crimes ~ prbarr + area"),
    as.formula("crimes ~ region"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ I(density*area) + region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ I(density*area) + region")))

SEP,MSEP,RMSEP
2217953579,246439287,15696.581
2194509525,243834392,15613.372
2170530515,241170057,15528.768
267308964,29700996,5369.317
225168185,25018687,4870.053
213026906,23669656,4681.167
2330862301,258984700,15848.010


In [20]:
eval(100, 9, 5, crimes, c(as.formula("crimes~region+area+I(sqrt(density))+I(sqrt(pctmin))+polpc+I(polpc^2)+I(log(prbarr))+I(sqrt(wcon))+wtrd+I(wtrd^2)+area:density+urban")))

SEP,MSEP,RMSEP
61489122,6832125,2536.939


In [21]:
eval_nb(100, 9, 5, crimes, c(as.formula("log(crimes)~region+area+I(sqrt(density))+I(sqrt(pctmin))+polpc+I(polpc^2)+I(log(prbarr))+I(sqrt(wcon))+wtrd+I(wtrd^2)+area:density+urban")))

SEP,MSEP,RMSEP
1109996849,123332983,10881.51


In [48]:
eval(100, 9, 5, crimes, c(
    as.formula("crimes ~ I(density*area) + region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ density*area + region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)"),
    as.formula("crimes ~ I(density*area) + region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)")))

SEP,MSEP,RMSEP
213026906,23669656,4681.167
201307444,22367494,4700.017
213026906,23669656,4681.167


In [63]:
glm(crimes ~ region, data=crimes, family=poisson(link="log"))


Call:  glm(formula = crimes ~ region, family = poisson(link = "log"), 
    data = crimes)

Coefficients:
(Intercept)  regionother   regionwest  
     8.4690      -0.7501      -1.5343  

Degrees of Freedom: 89 Total (i.e. Null);  87 Residual
Null Deviance:	    424300 
Residual Deviance: 350800 	AIC: 351600

In [4]:
eval(100, 9, 5, crimes, c(
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + polpc:area"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2) + polpc:area"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + area:density"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + area:density + polpc:area"),
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2) + area:density"),
    as.formula("crimes ~ I(density*area) + region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2)")))

ERROR: Error in eval(100, 9, 5, crimes, c(as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + polpc:area"), : unbenutzte Argumente (crimes, c(as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + polpc:area"), as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2) + polpc:area"), as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + area:density"), as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + area:density + polpc:area"), 
    as.formula("crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + wtrd + I(wtrd^2) + area:density"), as.formula("crimes ~ I(density*area) + reg

In [2]:
c(1,20)

[1]  1 20

In [70]:
anova(
    glm(crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + area:density, data=crimes, family=poisson(link="log")), 
    glm(crimes ~ region + area + I(sqrt(density)) + I(sqrt(pctmin)) + polpc + I(polpc^2) + I(log(prbarr)) + I(sqrt(wcon)) + wtrd + I(wtrd^2) + area:density + polpc:area, data=crimes, family=poisson(link="log")),
    test="LRT")

Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
77,13310.24,NA,NA,NA
76,13305.67,1,4.566245,0.03260796
